# Workspace Catalog Binding in Unity Catalog

**Objective:** Learn how to limit catalog access to specific Databricks workspaces to ensure environment isolation (e.g., preventing Dev data from being accessed in QA/Prod) while using a unified Metastore.

## 1. The Concept: Workspace-Catalog Binding

In a standard Unity Catalog setup, you might attach multiple workspaces (Development, QA, Production) to a single **Metastore**.
*   **Default Behavior:** By default, *all* catalogs created in the metastore are visible to *all* workspaces attached to it.
*   **The Problem:** You might see development catalogs (e.g., `dev_catalog`) appearing in your Production workspace. This creates clutter and potential governance risks.
*   **The Solution:** **Workspace Binding**. This allows admins to explicitly "bind" or "assign" a catalog to specific workspaces. If a catalog is not bound to a workspace, users in that workspace cannot see or access it, regardless of their permissions.

### Key Benefits
1.  **Environment Isolation:** Keeps Dev data in Dev, and Prod data in Prod.
2.  **Reduced Clutter:** Users only see catalogs relevant to their current environment.
3.  **Enhanced Security:** Adds an extra layer of access control at the workspace level.

## 2. Prerequisites for the Demo

To test this feature, the setup typically involves:
1.  **Two Workspaces:** E.g., `Databricks-Dev` and `Databricks-QA`.
2.  **Single Metastore:** Both workspaces must be attached to the **same** Unity Catalog Metastore (e.g., `azure-central-india`).
3.  **Admin Privileges:** You need Account Admin or Metastore Admin privileges to modify bindings.

*(Note: In the video, we use an existing Dev workspace and a newly created QA workspace tied to the same metastore.)*

## 3. Step-by-Step Implementation (UI)

### Step A: Verify Default Behavior
1.  Login to your **QA Workspace**.
2.  Navigate to **Catalog Explorer**.
3.  Observe that you can see `dev_catalog`, `test_db`, etc., which were created in the Dev environment.
4.  This confirms that currently, all workspaces have access to these catalogs.

### Step B: Restrict a Catalog (Binding)
1.  Switch to the **Dev Workspace** (or any workspace where you have admin rights).
2.  In **Catalog Explorer**, select the specific catalog you want to restrict (e.g., `dev`).
3.  Click on the **"Workspaces"** tab (next to Permissions).
4.  **Uncheck** the box "All workspaces have access".
5.  Click **"Assign to workspace"**.
6.  Select **only** the `Databricks-Dev` workspace from the list.
7.  Click **Assign**.

*Result: If you go back to the QA Workspace and refresh, the `dev` catalog will disappear. It is now bound exclusively to the Dev workspace.*

### Step C: Repeat for Other Catalogs
Repeat the process for other environment-specific catalogs (e.g., `dev_external`, `test_dev`) to completely clean up the QA environment view.

## 4. Sharing Data Across Workspaces (Common Catalog)

Sometimes you want a catalog to be available in **multiple** environments (e.g., a `common_data` catalog containing reference data).

1.  Create a new catalog named `common_catalog`.
2.  Go to the **"Workspaces"** tab for this catalog.
3.  Uncheck "All workspaces have access".
4.  Click **"Assign to workspace"**.
5.  Select **BOTH** `Databricks-Dev` and `Databricks-QA`.
6.  Click **Assign**.

*Result: This catalog will be visible in both workspaces, while the environment-specific catalogs remain hidden.*

## 5. Important Considerations

### Binding vs. Permissions
It is crucial to understand that Binding and Permissions work together but serve different purposes:

| Feature | Scope | Function |
| :--- | :--- | :--- |
| **Workspace Binding** | **Workspace Level** | Determines **visibility**. If a catalog is not bound to the workspace, no one inside that workspace can see it. |
| **Permissions (GRANT)** | **User/Group Level** | Determines **access**. Even if a catalog is bound to a workspace, a user still needs `USE CATALOG` and `SELECT` permissions to query data. |

**Best Practice:**
1.  Use **Binding** to ensure the catalog is available in the correct physical environment.
2.  Use **Permissions** (Unity Catalog Grants) to control which users within that environment can read/write data.

### Limitations
*   Binding is applied at the **Catalog** level. You cannot bind individual schemas or tables to specific workspaces.

In [ ]:
# Validation Check (Conceptual)
# While binding is a UI/Configuration action, you can use Python to verify 
# which catalog is visible in the current workspace.

# Run this code in your Dev Workspace, then run it in your QA Workspace.
# The list of catalogs should differ based on your binding configuration.

df_catalogs = spark.sql("SHOW CATALOGS")
display(df_catalogs)

## 6. Summary
By using Workspace Catalog Binding, we successfully:
1.  Restricted `dev` catalogs to the Dev Workspace.
2.  Hid unnecessary catalogs from the QA Workspace.
3.  Created a `common_catalog` shared between both.
4.  Ensured that metadata is governed centrally but accessed specifically.